In [6]:
import asyncio
import io
import glob
import os
import sys
import time
import uuid
import requests
from urllib.parse import urlparse
from io import BytesIO
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person
from tqdm import tqdm
import json

In [3]:
auth_file = "/home/ivan/pascal_adversarial_faces/azure_auth.json"
with open(auth_file, "r") as f:
    auth_data = json.loads(f.read())


In [4]:
face_client = FaceClient(
    auth_data["endpoint"], 
    CognitiveServicesCredentials(auth_data["key"])
)

In [5]:
# face_client.person_group.delete(person_group_id="robust_community_naive_mean_10_5_0p5")

In [7]:
person_id_to_name = {}
identities = []
for p in face_client.person_group_person.list(person_group_id="robust_community_naive_mean_10_5_0p5"):
    person_id_to_name[p.person_id] = p.name
    identities.append(p.name)

In [8]:
person_id_to_name

{'04559c68-5b9c-4b87-ae71-9366dbe00ec9': 'n002763',
 '04d620f7-082f-44e9-a105-da40cae0e30f': 'n007548',
 '182e323e-bbb2-41ae-a77e-b6ea1e6346c9': 'n004658',
 '3259200f-618d-43d1-92b2-88d81fddc909': 'n002503',
 '46e70ab8-60a7-4db8-a19e-20e4e62142e3': 'n009232',
 '536ad25d-1e33-46db-9295-623b1493b81e': 'n002647',
 '5878eec0-b8f1-43d2-badc-e6bfcb9e8297': 'n000958',
 '58df8bc9-cfda-4ea7-8e49-2db0aa6edd10': 'n003356',
 '6bb0ad38-d319-4db9-9e1b-feca2b2e0da5': 'n001683',
 '6cb77427-48ef-4e66-b367-63103c88cd27': 'n009288',
 '7b418734-efa9-4386-9d06-19e639a615ba': 'community_naive_mean_epsilon_0.1_png.tar.gz',
 '7f13ec76-964e-4574-a4bc-077786ed797f': 'community_naive_mean_epsilon_0.0_png.tar.gz',
 '90f8617b-04dc-40d4-b5e5-7f3e1154baaf': 'n008613',
 '99bfa98e-88d6-4c74-ac1a-c136f34581a4': 'n005303',
 'a0e1d88d-d6d8-485a-9273-4c3da8219978': 'n008655',
 'aa13a685-4b92-4391-8f5f-48001264175f': 'n001781',
 'bf774266-0c2d-41c6-ba71-1ab4ca33fa54': 'n005359',
 'ca225423-d5be-4ff1-b837-06501bf61c40': 'n0

In [13]:
def measure_azure_recall(
    face_client,
    image_directory,
    azure_person_group_name
):
    discovery = []
    true = []
    identified_as = []

    for protector in tqdm(identities):
        # We are sourcing query photos from epsilon_0.0.
        # In those cases, all subfolders in the "protected" identity have the same, clean
        # photo of the protector, so we just pick any single one that exists (e.g. n000958)
        # For the case where n000958 is itself the protector, n000958 is not present in its protected
        # subfolders, so we pick n000029 without loss of generality.
        if protector == "n000958":
            protected = "n000029"
        else:
            protected = "n000958"

        query_photos_paths = sorted(glob.glob(
            f"{image_directory}/{protector}/community_naive_mean/{protected}/epsilon_0.0/png/*"
        ))

        # For this person group, we picked the first 10 lexicographically sorted
        # photos to use in the lookup set, so for the query, we use the 11th and on.
        # (The query is not supposed to be in the lookup set).
        for i in range(11, len(query_photos_paths)):
            faces_in_query_photos = face_client.face.detect_with_stream(
                open(query_photos_paths[i], "r+b"), 
                detectionModel='detection_02'
            )
            if len(faces_in_query_photos) > 0:
                break


        # There should only be one face, so we use that as the query face.
        results = face_client.face.identify(
            [faces_in_query_photos[0].face_id],
            azure_person_group_name
        )
        
        true.append(protector)

        if len(results) < 1 or len(results[0].candidates) < 1:
            discovery.append(1.0)
            identified_as.append("None")

        else: 
            top_identity = person_id_to_name[results[0].candidates[0].person_id]

            identified_as.append(top_identity)
            
            # Note the switch of the term protector here:
            # protectors are also protected but we call them protectors because of the folder structure
            # In this case, the query photo belongs to the protector -- who is also protected by decoys 
            # of *other* protectors. Therefore, if the identity returned is that of the "protector,"
            # this is a failure in the defense.
            if top_identity == protector:
                discovery.append(1.0)
            else:
                discovery.append(0.0)

        time.sleep(10)
    
    for true_id, recognized_id in zip(true, identified_as):
        print(f"Face of {true_id} identitifed as {recognized_id}")
        
    return sum(discovery)/len(discovery)

In [ ]:
measure_azure_recall(
    face_client,
    "/data/vggface/test_perturbed_sampled",
    "robust_community_naive_mean_10_5_0p5"
)

 33%|███▎      | 7/21 [01:12<02:24, 10.35s/it]